# Group Seminar on Julia Pt. II  
## General Performance Tips / Pitfalls to avoid

<img src="http://imgs.xkcd.com/comics/optimization.png"></img>

## From input to machine code

<img src="./JuliaCompilation.png"></img>

Image from __De Sutter et al. (2016)__ https://arxiv.org/abs/1604.03410

## Measuring performance

### `@time` is your best friend.

In [5]:
@time begin
    local A = rand(250,250)
    local F = eigfact((A+A')/2)
end

  0.039608 seconds (45 allocations: 3.429 MB, 7.96% gc time)


Base.LinAlg.Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}([-6.26229,-6.15425,-6.08073,-5.95856,-5.80379,-5.73956,-5.65735,-5.58472,-5.55883,-5.42416  …  5.48082,5.54714,5.66972,5.74439,5.83742,5.92016,6.0536,6.22515,6.5032,125.419],250x250 Array{Float64,2}:
 -0.157852     0.0097549   -0.0919566   …   0.0596722   -0.0647217
  0.0160363   -0.041139     0.12211        -0.0190327   -0.061726 
  0.0589481   -0.0244084   -0.0391327       0.0359582   -0.0622407
 -0.0170758   -0.0401388    0.00173362     -0.065448    -0.0639973
  0.0181348   -0.00212726   0.0411502       0.0621008   -0.0606934
  0.0437556   -0.00176561   0.126031    …   0.0759294   -0.065033 
  0.0474396   -0.150214     0.0157082      -0.00318396  -0.0618228
  0.00443449   0.0907811   -0.145909       -0.0866613   -0.0601731
 -0.0340375   -0.0768437   -0.0891684       0.136947    -0.0648366
  0.0749697   -0.0727694   -0.0101764       0.0194892   -0.0629928
  0.0835363    0.00178625  -0.0697734   …  -0.00579012  -0.05

In [15]:
macro timeOften(ex::Expr,n::Int)
    :(begin 
     x=0
     @time for i = 1:$n
      $ex
     end
    end)
end

In [16]:
@timeOften rand() 100000

  0.000247 seconds


### More advanced: Profiling

Julia features a __statistical profiler__. Not every function call is back-traced.

In [6]:
function myEig(n::Int)
    A = rand(n,n)
    return eigfact((A+A')/2)
end

myEig (generic function with 1 method)

In [7]:
myEig(1);

In [8]:
Profile.clear()

In [9]:
@profile myEig(2500);

In [10]:
Profile.print(format=:flat)

 Count File                       Function                                 Line
   177 ....4/IJulia/src/IJulia.jl eventloop                                 143
   177 .../src/execute_request.jl execute_request_0x535c5df2                183
    15 In[6]                      myEig                                       2
   162 In[6]                      myEig                                       3
    21 array.jl                   reshape                                   148
    20 arraymath.jl               +                                          96
    50 arraymath.jl               ./                                         49
    28 arraymath.jl               transpose!                                323
     1 arraymath.jl               transposeblock!                           336
     2 arraymath.jl               transposeblock!                           339
    25 arraymath.jl               transposeblock!                           340
   113 arraymath.jl               transp

In [11]:
using ProfileView

INFO: Recompiling stale cache file /Network/Servers/mac25.thp.uni-koeln.de/Volumes/RAID/skleinbo/.julia/lib/v0.4/ProfileView.ji for module ProfileView.
INFO: Recompiling stale cache file /Network/Servers/mac25.thp.uni-koeln.de/Volumes/RAID/skleinbo/.julia/lib/v0.4/FixedPointNumbers.ji for module FixedPointNumbers.
INFO: Recompiling stale cache file /Network/Servers/mac25.thp.uni-koeln.de/Volumes/RAID/skleinbo/.julia/lib/v0.4/Colors.ji for module Colors.
INFO: Recompiling stale cache file /Network/Servers/mac25.thp.uni-koeln.de/Volumes/RAID/skleinbo/.julia/lib/v0.4/ColorTypes.ji for module ColorTypes.
INFO: Precompiling module ProfileViewSVG...


## 1. Type stability

In [13]:
function unstable(x::Int)
    if iseven(x)
        return Float64(x)
    else
        return Int64(x)
    end
end

unstable (generic function with 1 method)

In [14]:
typeof([ unstable(i) for i in 1:127 ])

Array{Union{Float64,Int64},1}

In [21]:
begin 
    local A = [ unstable(i) for i in 1:127 ]
    @timeOften A+1 100000
end

  0.682532 seconds (12.70 M allocations: 296.021 MB, 7.86% gc time)


In [24]:
begin 
    local A = [ Float64(i) for i in 1:127 ]
    @timeOften A+1 100000
end

  0.043505 seconds (100.00 k allocations: 103.760 MB, 55.21% gc time)


<hr />

In [25]:
function sum_unstable()
    sum = 0
    for i in 1:100
        sum += i/2
    end
    sum
end
function sum_stable()
    sum = 0.0
    for i in 1:100
        sum += i/2
    end
    sum
end

sum_stable (generic function with 1 method)

In [27]:
@timeOften sum_unstable() 100000;
@timeOften sum_stable() 100000;

  0.278849 seconds (20.00 M allocations: 305.176 MB, 21.57% gc time)
  0.006988 seconds


In [29]:
@code_warntype sum_unstable();

Variables:
  sum::ANY
  #s27::Int64
  i::Int64

Body:
  begin  # In[25], line 2:
      sum = 0 # In[25], line 3:
      GenSym(0) = $(Expr(:new, UnitRange{Int64}, 1, :(((top(getfield))(Base.Intrinsics,:select_value)::I)((Base.sle_int)(1,100)::Bool,100,(Base.box)(Int64,(Base.sub_int)(1,1)))::Int64)))
      #s27 = (top(getfield))(GenSym(0),:start)::Int64
      unless (Base.box)(Base.Bool,(Base.not_int)(#s27::Int64 === (Base.box)(Base.Int,(Base.add_int)((top(getfield))(GenSym(0),:stop)::Int64,1))::Bool)) goto 1
      2: 
      GenSym(2) = #s27::Int64
      GenSym(3) = (Base.box)(Base.Int,(Base.add_int)(#s27::Int64,1))
      i = GenSym(2)
      #s27 = GenSym(3) # In[25], line 4:
      sum = sum::UNION{FLOAT64,INT64} + (Base.box)(Base.Float64,(Base.div_float)((Base.box)(Float64,(Base.sitofp)(Float64,i::Int64)),(Base.box)(Float64,(Base.sitofp)(Float64,2))))::Float64
      3: 
      unless (Base.box)(Base.Bool,(Base.not_int)((Base.box)(Base.Bool,(Base.not_int)(#s27::Int64 === (Base.box)(Base.I

In [37]:
@code_llvm sum_unstable()


define %jl_value_t* @julia_sum_unstable_22318() {
top:
  %0 = alloca [5 x %jl_value_t*], align 8
  %.sub4 = bitcast [5 x %jl_value_t*]* %0 to %jl_value_t**
  %1 = getelementptr [5 x %jl_value_t*]* %0, i64 0, i64 2
  %2 = getelementptr [5 x %jl_value_t*]* %0, i64 0, i64 3
  store %jl_value_t* inttoptr (i64 6 to %jl_value_t*), %jl_value_t** %.sub4, align 8
  %3 = load %jl_value_t*** @jl_pgcstack, align 8
  %4 = getelementptr [5 x %jl_value_t*]* %0, i64 0, i64 1
  %.c = bitcast %jl_value_t** %3 to %jl_value_t*
  store %jl_value_t* %.c, %jl_value_t** %4, align 8
  store %jl_value_t** %.sub4, %jl_value_t*** @jl_pgcstack, align 8
  store %jl_value_t* null, %jl_value_t** %2, align 8
  %5 = getelementptr [5 x %jl_value_t*]* %0, i64 0, i64 4
  store %jl_value_t* null, %jl_value_t** %5, align 8
  store %jl_value_t* inttoptr (i64 4492828752 to %jl_value_t*), %jl_value_t** %1, align 8
  br label %L

L:                                                ; preds = %L, %top
  %lsr.iv = phi i64 [ 100, %t

## 2. Be careful with global variables.

In [207]:
# Alright
const global C = 100000
# Potentially VERY Bad
global D = 100000

function f(x)
    s=0
    for i in 1:10^6
        s += s+x+i + C end
    return s
end
function g(x)
    s=0
    for i in 1:10^6
        s += s+x+i + D end
    return s
end

g (generic function with 2 methods)

In [209]:
@time f(10);

  0.000475 seconds (5 allocations: 176 bytes)


In [211]:
@time g(10);

  0.077768 seconds (4.00 M allocations: 61.028 MB, 5.17% gc time)


In [232]:
function g(x,y::Int)
    s=0
    for i in 1:Int(1e6)
        s+=s+x+i + y end
    return s
end

g (generic function with 2 methods)

In [239]:
@time g(10,D);

  0.000460 seconds (5 allocations: 176 bytes)


__Q: Why is a `const` so much more efficient?__  
A: The compiler cannot infer the type of a dynamic global. A constants type is fixed at declaration and can be inlined.

In [215]:
@code_llvm f(10)


define i64 @julia_f_4061(i64) {
top:
  %1 = call i64 @julia_power_by_squaring_115(i64 10, i64 6)
  %2 = icmp sgt i64 %1, 0
  %3 = select i1 %2, i64 %1, i64 0
  %4 = icmp eq i64 %3, 0
  br i1 %4, label %L3, label %L.preheader

L.preheader:                                      ; preds = %top
  %5 = add i64 %0, 100000
  br label %L

L:                                                ; preds = %L, %L.preheader
  %s.0 = phi i64 [ %8, %L ], [ 0, %L.preheader ]
  %"#s749.0" = phi i64 [ %6, %L ], [ 1, %L.preheader ]
  %6 = add i64 %"#s749.0", 1
  %factor = shl i64 %s.0, 1
  %7 = add i64 %5, %"#s749.0"
  %8 = add i64 %7, %factor
  %9 = icmp eq i64 %"#s749.0", %3
  br i1 %9, label %L3, label %L

L3:                                               ; preds = %L, %top
  %s.1 = phi i64 [ 0, %top ], [ %8, %L ]
  ret i64 %s.1
}


In [216]:
@code_llvm g(10)


define %jl_value_t* @julia_g_4062(i64) {
top:
  %1 = alloca [8 x %jl_value_t*], align 8
  %.sub = getelementptr inbounds [8 x %jl_value_t*]* %1, i64 0, i64 0
  %2 = getelementptr [8 x %jl_value_t*]* %1, i64 0, i64 2
  %3 = getelementptr [8 x %jl_value_t*]* %1, i64 0, i64 5
  store %jl_value_t* inttoptr (i64 12 to %jl_value_t*), %jl_value_t** %.sub, align 8
  %4 = getelementptr [8 x %jl_value_t*]* %1, i64 0, i64 1
  %5 = load %jl_value_t*** @jl_pgcstack, align 8
  %.c = bitcast %jl_value_t** %5 to %jl_value_t*
  store %jl_value_t* %.c, %jl_value_t** %4, align 8
  store %jl_value_t** %.sub, %jl_value_t*** @jl_pgcstack, align 8
  %6 = getelementptr [8 x %jl_value_t*]* %1, i64 0, i64 3
  store %jl_value_t* null, %jl_value_t** %6, align 8
  %7 = getelementptr [8 x %jl_value_t*]* %1, i64 0, i64 4
  store %jl_value_t* null, %jl_value_t** %7, align 8
  store %jl_value_t* null, %jl_value_t** %3, align 8
  %8 = getelementptr [8 x %jl_value_t*]* %1, i64 0, i64 6
  store %jl_value_t* null, %jl_va

In [217]:
@code_llvm g(10,D)


define i64 @julia_g_4063(i64, i64) {
top:
  %2 = add i64 %1, %0
  br label %L

L:                                                ; preds = %L, %top
  %s.0 = phi i64 [ 0, %top ], [ %5, %L ]
  %"#s767.0" = phi i64 [ 1, %top ], [ %3, %L ]
  %3 = add i64 %"#s767.0", 1
  %factor = shl i64 %s.0, 1
  %4 = add i64 %2, %"#s767.0"
  %5 = add i64 %4, %factor
  %6 = icmp eq i64 %"#s767.0", 1000000
  br i1 %6, label %L3, label %L

L3:                                               ; preds = %L
  ret i64 %5
}


<span style="font-size:14pt">$\Longrightarrow$ Adopt a __functional__ programming style!</span>  



## 3. Respect the order.
### Unlike C, Julia has _column major_ order!

In [240]:
function add_matrices_one(A::Matrix, B::Matrix)
    C = Matrix{eltype(A)}(size(A)...)
    for i = 1:size(A,1)
        for j = 1:size(A,2)
            C[i,j] = A[i,j] + B[i,j]
        end
    end
    return C
end
function add_matrices_two(A::Matrix, B::Matrix)
    C = Matrix{eltype(A)}(size(A)...)
    for i = 1:size(A,2)
        for j = 1:size(A,1)
            C[j,i] = A[j,i] + B[j,i]
        end
    end
    return C
end

add_matrices_two (generic function with 1 method)

In [241]:
A = rand(1000,1000);
B = rand(1000,1000);

In [243]:
@time add_matrices_one(A,B);
@time add_matrices_two(A,B);

  0.015843 seconds (6 allocations: 7.630 MB)
  0.003286 seconds (6 allocations: 7.630 MB)


__Note:__ Don't bother implementing LinAlg operations.

In [244]:
@time A+B;

  0.002716 seconds (9 allocations: 7.630 MB, 139.10% gc time)


## 4. Disable safty nets where appropriate.

By default Julia performs __out-of-bound checks__ on array access. We get an exception without crashing the kernel.

In [245]:
add_matrices_two(rand(4,4),rand(3,3))

LoadError: LoadError: BoundsError: attempt to access 3x3 Array{Float64,2}:
 0.0570808  0.0362426  0.780516
 0.294962   0.668098   0.144707
 0.667597   0.594511   0.917051
  at index [4,1]
while loading In[245], in expression starting on line 1

However, this can cost performance and is unneccessary if we guarantee inbounds access $\rightarrow$ __@inbounds__

In [246]:
function add_matrices_three(A::Matrix, B::Matrix)
    C = Matrix{eltype(A)}(size(A)...)
    if size(A) != size(B)
        return nothing
    end
    @inbounds for i = 1:size(A,2)
        for j = 1:size(A,1)
           C[j,i] = A[j,i] + B[j,i]
        end
    end
    return C
end

add_matrices_three (generic function with 1 method)

In [248]:
@time add_matrices_three(A,B);

  0.002569 seconds (6 allocations: 7.630 MB)


## 5. Pre-allocate output

_[Verbatim from http://docs.julialang.org/en/release-0.4/manual/performance-tips/#pre-allocating-outputs]_

In [74]:
function xinc(x)
    return [x, x+1, x+2]
end

function loopinc()
    y = 0
    for i = 1:10^7
        ret = xinc(i)
        y += ret[2]
    end
    y
end

loopinc (generic function with 1 method)

Every call to `xinc` allocates __a new array__.   
$\rightarrow$ Better allocate the array beforehand and __update it__.

In [72]:
function xinc!{T}(ret::AbstractVector{T}, x::T)
    ret[1] = x
    ret[2] = x+1
    ret[3] = x+2
    nothing
end

function loopinc_prealloc()
    ret = Array(Int, 3)
    y = 0
    for i = 1:10^7
        xinc!(ret, i)
        y += ret[2]
    end
    y
end

loopinc_prealloc (generic function with 1 method)

In [76]:
@time loopinc();
@time loopinc_prealloc();

  0.510639 seconds (40.00 M allocations: 1.341 GB, 13.42% gc time)
  0.025712 seconds (6 allocations: 272 bytes)
